## Implementing a ChatGPT App with LangChain from Scratch

In [ ]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True) 

from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_classic.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate


llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1)

prompt = ChatPromptTemplate(
    input_variables=["content"],
    messages=[
#         SystemMessage(content="You are a chatbot having a conversation with a human."),
        SystemMessage(content='You respond only in German.'),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True
)

while True:
    content = input('Your prompt: ')
    if content.lower() in ['quit', 'exit', 'bye']:
        print('Goodbye!')
        break
    
    response = chain.invoke({'content': content})
    print(response)
    print('-' * 50)

{'content': 'hello g kia hal haio?', 'text': 'Hallo! Mir geht es gut, danke. Wie kann ich Ihnen helfen?'}
--------------------------------------------------
Goodbye!


## Adding Chat Memory Using ConversationBufferMemory

In [ ]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=False) 

from langchain_openai import ChatOpenAI
from langchain_classic.chains import LLMChain, SimpleSequentialChain
from langchain_core.messages import SystemMessage

# 1. Imports
from langchain_classic.memory import ConversationBufferMemory
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1)

# 2. Create memory
memory = ConversationBufferMemory(
    memory_key='chat_history',
    return_messages=True
)

# 3. add  MessagesPlaceholder(variable_name='messages') to the prompt
prompt = ChatPromptTemplate(
    input_variables=["content", "chat_history"],
    messages=[
        SystemMessage(content="You are a chatbot having a conversation with a human."),
        MessagesPlaceholder(variable_name="chat_history"), # Where the memory will be stored.
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

# 4. Add the memory to the chain
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False
)

while True:
    content = input('Your prompt: ')
    if content.lower() in ['quit', 'exit', 'bye']:
        print('Goodbye!')
        break
    
    response = chain.invoke({'content': content})
    print(response)
    print('-' * 50)

Goodbye!


## Saving Chat Sessions

In [3]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=False) 

from langchain_openai import ChatOpenAI
# Messages and Prompts come from 'core'
from langchain_core.messages import SystemMessage
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

# Legacy chains and memory come from 'classic'
from langchain_classic.chains import LLMChain
from langchain_classic.memory import ConversationBufferMemory, FileChatMessageHistory

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1)

# 2. Add an additional keyword argument to the ConversationBufferMemory() constructor
history = FileChatMessageHistory('chat_history.json')
memory = ConversationBufferMemory(
    memory_key='chat_history',
    chat_memory=history,
    return_messages=True
)

prompt = ChatPromptTemplate(
    input_variables=["content", "chat_history"],
    messages=[
        SystemMessage(content="You are a chatbot having a conversation with a human."),
        MessagesPlaceholder(variable_name="chat_history"), 
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False
)

while True:
    content = input('Your prompt: ')
    if content.lower() in ['quit', 'exit', 'bye']:
        print('Goodbye!')
        break
    
    response = chain.invoke({'content': content})
    print(response)
    print('-' * 50)

{'content': 'hello g kia hal hai', 'chat_history': [], 'text': 'Hello! Main theek hoon, shukriya. Aap kaise hain?'}
--------------------------------------------------
{'content': 'mai theek or sunao kesi guzr rahi?', 'chat_history': [HumanMessage(content='hello g kia hal hai', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello! Main theek hoon, shukriya. Aap kaise hain?', additional_kwargs={}, response_metadata={})], 'text': 'Mai bhi theek hoon, shukriya. Aapne kuch interesting kiya ya dekha aaj kal?'}
--------------------------------------------------
{'content': 'hassan pagal hai mera dost uska kia kru', 'chat_history': [HumanMessage(content='hello g kia hal hai', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello! Main theek hoon, shukriya. Aap kaise hain?', additional_kwargs={}, response_metadata={}), HumanMessage(content='mai theek or sunao kesi guzr rahi?', additional_kwargs={}, response_metadata={}), AIMessage(content='Mai bhi theek hoon, sh

In [4]:
# The messages property contains the list of messages in order.
print(history.messages)

[HumanMessage(content='hello g kia hal hai', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello! Main theek hoon, shukriya. Aap kaise hain?', additional_kwargs={}, response_metadata={}), HumanMessage(content='mai theek or sunao kesi guzr rahi?', additional_kwargs={}, response_metadata={}), AIMessage(content='Mai bhi theek hoon, shukriya. Aapne kuch interesting kiya ya dekha aaj kal?', additional_kwargs={}, response_metadata={}), HumanMessage(content='hassan pagal hai mera dost uska kia kru', additional_kwargs={}, response_metadata={}), AIMessage(content='Agar aapka dost Hassan ko lagta hai ki woh pagal hai, to shayad usse achhe se baat karke uska mood samjha ja sakta hai. Pyaar aur samajh se hi kisi dost ko samjha ja sakta hai. Aap usse uske feelings share karne ke liye encourage kar sakte hain. Kabhi kabhi humein apne dosto ki madad karne ki zarurat hoti hai.', additional_kwargs={}, response_metadata={})]
